커널사이즈 수정 ,
맥스풀링 삭제,
레이어 삭제하고 vgg11비슷하게,
dropout 삭제,
중간에 batch normal 넣는거 vs 안넣는거

1. 커널 사이즈 수정 2x2

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.conv5 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv16 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dense2 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs, training=training)
        net = self.conv2(net, training=training)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.conv5(net, training=training)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.conv16(net, training=training)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.batchnorm(net)
        net = self.dense2(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()

    # data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                                     batch_size=batch_size), 
                        epochs=training_epochs, 
                        verbose=2, 
                        callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
                        steps_per_epoch=train_images.shape[0] // batch_size,
                        validation_data=(val_images, val_labels))


2.10.0
2.10.0
Epoch 1/2500
Epoch 1/2500 - Train Accuracy: 0.3969 - Val Accuracy: 0.3944 - Test Accuracy: 0.4032
312/312 - 144s - loss: 12.5994 - accuracy: 0.3315 - val_loss: 9.6911 - val_accuracy: 0.3944 - lr: 0.1000 - 144s/epoch - 462ms/step
Epoch 2/2500
Epoch 2/2500 - Train Accuracy: 0.3552 - Val Accuracy: 0.3530 - Test Accuracy: 0.3584
312/312 - 128s - loss: 7.5228 - accuracy: 0.4536 - val_loss: 6.1274 - val_accuracy: 0.3530 - lr: 0.1000 - 128s/epoch - 411ms/step
Epoch 3/2500
Epoch 3/2500 - Train Accuracy: 0.3952 - Val Accuracy: 0.3953 - Test Accuracy: 0.3828
312/312 - 128s - loss: 4.6783 - accuracy: 0.5115 - val_loss: 4.1460 - val_accuracy: 0.3953 - lr: 0.1000 - 128s/epoch - 410ms/step
Epoch 4/2500
Epoch 4/2500 - Train Accuracy: 0.4530 - Val Accuracy: 0.4439 - Test Accuracy: 0.4549
312/312 - 128s - loss: 3.1654 - accuracy: 0.5561 - val_loss: 3.0853 - val_accuracy: 0.4439 - lr: 0.1000 - 128s/epoch - 411ms/step
Epoch 5/2500
Epoch 5/2500 - Train Accuracy: 0.5237 - Val Accuracy: 0.5117

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/vgg16_model_2/dense_4/MatMul/MatMul_1' defined at (most recent call last):
    File "C:\Users\User\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\User\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\User\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\User\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 701, in start
      self.io_loop.start()
    File "C:\Users\User\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\User\anaconda3\lib\asyncio\windows_events.py", line 321, in run_foreve

2. bn제거

In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv5 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv16 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.dense3 = keras.layers.Dense(units=10)  # 변경된 부분: units=10
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs, training=training)
        net = self.conv2(net, training=training)
        net = self.maxPooling1(net)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.maxPooling2(net)
        net = self.conv5(net, training=training)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.maxPooling3(net)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.maxPooling5(net)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.conv16(net, training=training)
        net = self.maxPooling6(net)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.dense3(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()

    # data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit_generator(datagen.flow(train_images, train_labels,
                                     batch_size=batch_size), 
                        epochs=training_epochs, 
                        verbose=2, 
                        callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
                        steps_per_epoch=train_images.shape[0] // batch_size,
                        validation_data=(val_images, val_labels))


2.10.0
2.10.0


C:\Users\User\AppData\Local\Temp\ipykernel_7552\2296324173.py:170: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(train_images, train_labels,


Epoch 1/2500
Epoch 1/2500 - Train Accuracy: 0.1681 - Val Accuracy: 0.1711 - Test Accuracy: 0.1663
312/312 - 25s - loss: 15.3884 - accuracy: 0.1865 - val_loss: 12.2614 - val_accuracy: 0.1711 - lr: 0.1000 - 25s/epoch - 79ms/step
Epoch 2/2500
Epoch 2/2500 - Train Accuracy: 0.1599 - Val Accuracy: 0.1668 - Test Accuracy: 0.1671
312/312 - 22s - loss: 9.3646 - accuracy: 0.2746 - val_loss: 7.8426 - val_accuracy: 0.1668 - lr: 0.1000 - 22s/epoch - 71ms/step
Epoch 3/2500
Epoch 3/2500 - Train Accuracy: 0.2077 - Val Accuracy: 0.2083 - Test Accuracy: 0.2051
312/312 - 22s - loss: 5.8377 - accuracy: 0.3283 - val_loss: 5.7074 - val_accuracy: 0.2083 - lr: 0.1000 - 22s/epoch - 72ms/step
Epoch 4/2500
Epoch 4/2500 - Train Accuracy: 0.4217 - Val Accuracy: 0.4233 - Test Accuracy: 0.4236
312/312 - 23s - loss: 3.9215 - accuracy: 0.3941 - val_loss: 3.2750 - val_accuracy: 0.4233 - lr: 0.1000 - 23s/epoch - 74ms/step
Epoch 5/2500
Epoch 5/2500 - Train Accuracy: 0.4241 - Val Accuracy: 0.4230 - Test Accuracy: 0.4246


KeyboardInterrupt: 

dense layer 2개without bn

In [29]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv5 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv16 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.dense2 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.dense3 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs, training=training)
        net = self.conv2(net, training=training)
        net = self.maxPooling1(net)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.maxPooling2(net)
        net = self.conv5(net, training=training)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.maxPooling3(net)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.maxPooling5(net)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.conv16(net, training=training)
        net = self.maxPooling6(net)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.dense3(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()

    # data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                        batch_size=batch_size), 
          epochs=training_epochs, 
          verbose=0,  # 출력 숨기기
          callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
          steps_per_epoch=train_images.shape[0] // batch_size,
          validation_data=(val_images, val_labels))


2.10.0
2.10.0
Epoch 1/2500 - Train Accuracy: 0.1879 - Val Accuracy: 0.1885 - Test Accuracy: 0.1936
Epoch 2/2500 - Train Accuracy: 0.1664 - Val Accuracy: 0.1654 - Test Accuracy: 0.1706
Epoch 3/2500 - Train Accuracy: 0.2132 - Val Accuracy: 0.2050 - Test Accuracy: 0.2132
Epoch 4/2500 - Train Accuracy: 0.3612 - Val Accuracy: 0.3484 - Test Accuracy: 0.3558
Epoch 5/2500 - Train Accuracy: 0.4691 - Val Accuracy: 0.4656 - Test Accuracy: 0.4625
Epoch 6/2500 - Train Accuracy: 0.5115 - Val Accuracy: 0.5007 - Test Accuracy: 0.5073
Epoch 7/2500 - Train Accuracy: 0.5777 - Val Accuracy: 0.5712 - Test Accuracy: 0.5719
Epoch 8/2500 - Train Accuracy: 0.6317 - Val Accuracy: 0.6310 - Test Accuracy: 0.6239
Epoch 9/2500 - Train Accuracy: 0.3721 - Val Accuracy: 0.3739 - Test Accuracy: 0.3786
Epoch 10/2500 - Train Accuracy: 0.6155 - Val Accuracy: 0.6090 - Test Accuracy: 0.6005
Epoch 11/2500 - Train Accuracy: 0.5906 - Val Accuracy: 0.5877 - Test Accuracy: 0.5774
Epoch 12/2500 - Train Accuracy: 0.6201 - Val Accu

KeyboardInterrupt: 

##이거

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.conv5 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv8 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv9 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv10 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dense3 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs, training=training)
        net = self.conv2(net, training=training)
        net = self.maxPooling1(net)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.maxPooling2(net)
        net = self.conv5(net, training=training)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.maxPooling3(net)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.maxPooling5(net)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.conv16(net, training=training)
        net = self.maxPooling6(net)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.batchnorm(net)
        net = self.dense3(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()

    # data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                        batch_size=batch_size), 
          epochs=training_epochs, 
          verbose=0,  # 출력 숨기기
          callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
          steps_per_epoch=train_images.shape[0] // batch_size,
          validation_data=(val_images, val_labels))


3. 출력 뉴런 감소

2.10.0
2.10.0
Epoch 1/2500
Epoch 1/2500 - Train Accuracy: 0.4072 - Val Accuracy: 0.4122 - Test Accuracy: 0.4129
312/312 - 124s - loss: 12.0520 - accuracy: 0.3500 - val_loss: 9.3524 - val_accuracy: 0.4122 - lr: 0.1000 - 124s/epoch - 396ms/step
Epoch 2/2500
Epoch 2/2500 - Train Accuracy: 0.4450 - Val Accuracy: 0.4491 - Test Accuracy: 0.4400
312/312 - 122s - loss: 7.2676 - accuracy: 0.4595 - val_loss: 5.7114 - val_accuracy: 0.4491 - lr: 0.1000 - 122s/epoch - 391ms/step
Epoch 3/2500
Epoch 3/2500 - Train Accuracy: 0.4602 - Val Accuracy: 0.4539 - Test Accuracy: 0.4519
312/312 - 122s - loss: 4.5406 - accuracy: 0.5130 - val_loss: 3.8828 - val_accuracy: 0.4539 - lr: 0.1000 - 122s/epoch - 391ms/step
Epoch 4/2500
Epoch 4/2500 - Train Accuracy: 0.5299 - Val Accuracy: 0.5264 - Test Accuracy: 0.5232
312/312 - 122s - loss: 3.0693 - accuracy: 0.5545 - val_loss: 2.7438 - val_accuracy: 0.5264 - lr: 0.1000 - 122s/epoch - 392ms/step
Epoch 5/2500
Epoch 5/2500 - Train Accuracy: 0.5598 - Val Accuracy: 0.5514

KeyboardInterrupt: 

4.맥스풀링 삭제

In [25]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.conv5 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv16 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dense2 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs, training=training)
        net = self.conv2(net, training=training)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.conv5(net, training=training)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.conv16(net, training=training)
        net = self.flat(net)
        net = self.dropOut(net)
        net = self.dense1(net)
        net = self.batchnorm(net)
        net = self.dense2(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20
 
    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()

    # data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                        batch_size=batch_size), 
          epochs=training_epochs, 
          verbose=0,  # 출력 숨기기
          callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
          steps_per_epoch=train_images.shape[0] // batch_size,
          validation_data=(val_images, val_labels))


2.10.0
2.10.0
Epoch 1/2500 - Train Accuracy: 0.3354 - Val Accuracy: 0.3359 - Test Accuracy: 0.3361
Epoch 2/2500 - Train Accuracy: 0.2933 - Val Accuracy: 0.2985 - Test Accuracy: 0.2952
Epoch 3/2500 - Train Accuracy: 0.4087 - Val Accuracy: 0.4081 - Test Accuracy: 0.4089
Epoch 4/2500 - Train Accuracy: 0.2959 - Val Accuracy: 0.2961 - Test Accuracy: 0.2861
Epoch 5/2500 - Train Accuracy: 0.5599 - Val Accuracy: 0.5509 - Test Accuracy: 0.5485
Epoch 6/2500 - Train Accuracy: 0.6143 - Val Accuracy: 0.5982 - Test Accuracy: 0.6027
Epoch 7/2500 - Train Accuracy: 0.6537 - Val Accuracy: 0.6437 - Test Accuracy: 0.6416
Epoch 8/2500 - Train Accuracy: 0.6789 - Val Accuracy: 0.6676 - Test Accuracy: 0.6582
Epoch 9/2500 - Train Accuracy: 0.6982 - Val Accuracy: 0.6851 - Test Accuracy: 0.6794
Epoch 10/2500 - Train Accuracy: 0.7179 - Val Accuracy: 0.7013 - Test Accuracy: 0.7005
Epoch 11/2500 - Train Accuracy: 0.7179 - Val Accuracy: 0.7083 - Test Accuracy: 0.7016
Epoch 12/2500 - Train Accuracy: 0.7047 - Val Accu

KeyboardInterrupt: 

5. 레이어 2개씩으로 맞춤

In [14]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv5 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dense2 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs, training=training)
        net = self.conv2(net, training=training)
        net = self.maxPooling1(net)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.maxPooling2(net)
        net = self.conv5(net, training=training)
        net = self.conv6(net, training=training)
        net = self.maxPooling3(net)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.maxPooling5(net)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.maxPooling6(net)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.batchnorm(net)
        net = self.dense2(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20
 
    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()

    # data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                        batch_size=batch_size), 
          epochs=training_epochs, 
          verbose=0,  # 출력 숨기기
          callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
          steps_per_epoch=train_images.shape[0] // batch_size,
          validation_data=(val_images, val_labels))


2.10.0
2.10.0
Epoch 1/2500 - Train Accuracy: 0.1357 - Val Accuracy: 0.1305 - Test Accuracy: 0.1372
Epoch 2/2500 - Train Accuracy: 0.2082 - Val Accuracy: 0.2012 - Test Accuracy: 0.2109
Epoch 3/2500 - Train Accuracy: 0.2591 - Val Accuracy: 0.2576 - Test Accuracy: 0.2601
Epoch 4/2500 - Train Accuracy: 0.5395 - Val Accuracy: 0.5292 - Test Accuracy: 0.5278
Epoch 5/2500 - Train Accuracy: 0.4752 - Val Accuracy: 0.4709 - Test Accuracy: 0.4743
Epoch 6/2500 - Train Accuracy: 0.6534 - Val Accuracy: 0.6476 - Test Accuracy: 0.6474
Epoch 7/2500 - Train Accuracy: 0.6302 - Val Accuracy: 0.6244 - Test Accuracy: 0.6185
Epoch 8/2500 - Train Accuracy: 0.6758 - Val Accuracy: 0.6705 - Test Accuracy: 0.6607
Epoch 9/2500 - Train Accuracy: 0.6760 - Val Accuracy: 0.6728 - Test Accuracy: 0.6662
Epoch 10/2500 - Train Accuracy: 0.7306 - Val Accuracy: 0.7173 - Test Accuracy: 0.7110
Epoch 11/2500 - Train Accuracy: 0.7047 - Val Accuracy: 0.6958 - Test Accuracy: 0.6876
Epoch 12/2500 - Train Accuracy: 0.7510 - Val Accu

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [23]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # GPU 메모리 즉시 사용 설정
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('GPU 메모리 즉시 사용이 활성화되었습니다.')
    except RuntimeError as e:
        # 프로그램 실행 중 에러 발생 시
        print(e)

def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')
class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv16 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.dense3 = keras.layers.Dense(units=10)  # 변경된 부분: units=10
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs, training=training)
        net = self.conv2(net, training=training)
        net = self.maxPooling1(net)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.maxPooling2(net)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.maxPooling3(net)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.maxPooling5(net)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.conv16(net, training=training)
        net = self.maxPooling6(net)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.dense3(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()

    # data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                        batch_size=batch_size), 
          epochs=training_epochs, 
          verbose=0,  # 출력 숨기기
          callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
          steps_per_epoch=train_images.shape[0] // batch_size,
          validation_data=(val_images, val_labels))

Physical devices cannot be modified after being initialized
2.10.0
2.10.0
Epoch 1/2500 - Train Accuracy: 0.1546 - Val Accuracy: 0.1582 - Test Accuracy: 0.1596
Epoch 2/2500 - Train Accuracy: 0.2333 - Val Accuracy: 0.2271 - Test Accuracy: 0.2248
Epoch 3/2500 - Train Accuracy: 0.2483 - Val Accuracy: 0.2398 - Test Accuracy: 0.2422
Epoch 4/2500 - Train Accuracy: 0.3700 - Val Accuracy: 0.3744 - Test Accuracy: 0.3730
Epoch 5/2500 - Train Accuracy: 0.3952 - Val Accuracy: 0.3952 - Test Accuracy: 0.3979
Epoch 6/2500 - Train Accuracy: 0.5253 - Val Accuracy: 0.5183 - Test Accuracy: 0.5222
Epoch 7/2500 - Train Accuracy: 0.1270 - Val Accuracy: 0.1291 - Test Accuracy: 0.1275
Epoch 8/2500 - Train Accuracy: 0.6590 - Val Accuracy: 0.6550 - Test Accuracy: 0.6510
Epoch 9/2500 - Train Accuracy: 0.6557 - Val Accuracy: 0.6526 - Test Accuracy: 0.6545
Epoch 10/2500 - Train Accuracy: 0.7061 - Val Accuracy: 0.7016 - Test Accuracy: 0.6973
Epoch 11/2500 - Train Accuracy: 0.6145 - Val Accuracy: 0.6052 - Test Accura

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# GPU 메모리 즉시 사용 활성화
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # GPU 메모리 즉시 사용 설정
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('GPU 메모리 즉시 사용이 활성화되었습니다.')
    except RuntimeError as e:
        # 프로그램 실행 중 에러 발생 시
        print(e)


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels
class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')

class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv5 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dense2 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv1(inputs, training=training)
        net = self.conv2(net, training=training)
        net = self.maxPooling1(net)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.maxPooling2(net)
        net = self.conv5(net, training=training)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.maxPooling3(net)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.maxPooling5(net)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.maxPooling6(net)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.batchnorm(net)
        net = self.dense2(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()


    # data augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                        batch_size=batch_size), 
          epochs=training_epochs, 
          verbose=0,  # 출력 숨기기
          callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
          steps_per_epoch=train_images.shape[0] // batch_size,
          validation_data=(val_images, val_labels))

Physical devices cannot be modified after being initialized
2.10.0
2.10.0
Epoch 1/2500 - Train Accuracy: 0.1477 - Val Accuracy: 0.1476 - Test Accuracy: 0.1448
Epoch 2/2500 - Train Accuracy: 0.3149 - Val Accuracy: 0.3169 - Test Accuracy: 0.3108
Epoch 3/2500 - Train Accuracy: 0.4530 - Val Accuracy: 0.4480 - Test Accuracy: 0.4466
Epoch 4/2500 - Train Accuracy: 0.4403 - Val Accuracy: 0.4389 - Test Accuracy: 0.4376
Epoch 5/2500 - Train Accuracy: 0.5944 - Val Accuracy: 0.5923 - Test Accuracy: 0.5890
Epoch 6/2500 - Train Accuracy: 0.6265 - Val Accuracy: 0.6202 - Test Accuracy: 0.6203
Epoch 7/2500 - Train Accuracy: 0.6974 - Val Accuracy: 0.6903 - Test Accuracy: 0.6834
Epoch 8/2500 - Train Accuracy: 0.6967 - Val Accuracy: 0.6886 - Test Accuracy: 0.6854
Epoch 9/2500 - Train Accuracy: 0.7329 - Val Accuracy: 0.7289 - Test Accuracy: 0.7294
Epoch 10/2500 - Train Accuracy: 0.7638 - Val Accuracy: 0.7623 - Test Accuracy: 0.7538
Epoch 11/2500 - Train Accuracy: 0.7857 - Val Accuracy: 0.7782 - Test Accura

KeyboardInterrupt: 

layer 추가

In [28]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# GPU 메모리 즉시 사용 활성화
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # GPU 메모리 즉시 사용 설정
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('GPU 메모리 즉시 사용이 활성화되었습니다.')
    except RuntimeError as e:
        # 프로그램 실행 중 에러 발생 시
        print(e)


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels
class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')

class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv0 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3])
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv5 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv8 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv9 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dense2 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv0(inputs, training=training)
        net = self.conv1(net, training=training)
        net = self.conv2(net, training=training)
        net = self.maxPooling1(net)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.conv5(net, training=training)
        net = self.maxPooling2(net)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.conv8(net, training=training)
        net = self.maxPooling3(net)
        net = self.conv9(net, training=training)
        net = self.conv11(net, training=training)
        net = self.maxPooling5(net)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.maxPooling6(net)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.batchnorm(net)
        net = self.dense2(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 2500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()


    # data augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                        batch_size=batch_size), 
          epochs=training_epochs, 
          verbose=0,  # 출력 숨기기
          callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
          steps_per_epoch=train_images.shape[0] // batch_size,
          validation_data=(val_images, val_labels))

Physical devices cannot be modified after being initialized
2.10.0
2.10.0
Epoch 1/2500 - Train Accuracy: 0.1619 - Val Accuracy: 0.1603 - Test Accuracy: 0.1665
Epoch 2/2500 - Train Accuracy: 0.1510 - Val Accuracy: 0.1460 - Test Accuracy: 0.1502
Epoch 3/2500 - Train Accuracy: 0.2246 - Val Accuracy: 0.2215 - Test Accuracy: 0.2264
Epoch 4/2500 - Train Accuracy: 0.4171 - Val Accuracy: 0.4123 - Test Accuracy: 0.4147
Epoch 5/2500 - Train Accuracy: 0.5199 - Val Accuracy: 0.5127 - Test Accuracy: 0.5104
Epoch 6/2500 - Train Accuracy: 0.6073 - Val Accuracy: 0.6071 - Test Accuracy: 0.5931


KeyboardInterrupt: 

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# GPU 메모리 즉시 사용 활성화
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        # GPU 메모리 즉시 사용 설정
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print('GPU 메모리 즉시 사용이 활성화되었습니다.')
    except RuntimeError as e:
        # 프로그램 실행 중 에러 발생 시
        print(e)


def normalization(train_images, test_images):
    mean = np.mean(train_images, axis=(0, 1, 2, 3))
    std = np.std(train_images, axis=(0, 1, 2, 3))
    train_images = (train_images - mean) / (std + 1e-7)
    test_images = (test_images - mean) / (std + 1e-7)
    return train_images, test_images


def load_images():
    (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

    train_images = train_images.astype(np.float32)
    test_images = test_images.astype(np.float32)

    (train_images, test_images) = normalization(train_images, test_images)

    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    # Randomly select 20% of the training data as validation data
    validation_split = 0.2
    split_index = int(train_images.shape[0] * (1 - validation_split))
    train_images, val_images = train_images[:split_index], train_images[split_index:]
    train_labels, val_labels = train_labels[:split_index], train_labels[split_index:]

    return train_images, train_labels, val_images, val_labels, test_images, test_labels
class EpochEndCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_data, val_data, test_data):
        super(EpochEndCallback, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs=None):
        train_loss, train_accuracy = self.model.evaluate(self.train_data[0], self.train_data[1], verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.val_data[0], self.val_data[1], verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.test_data[0], self.test_data[1], verbose=0)
        
        print(f'Epoch {epoch+1}/{training_epochs} - '
              f'Train Accuracy: {train_accuracy:.4f} - '
              f'Val Accuracy: {val_accuracy:.4f} - '
              f'Test Accuracy: {test_accuracy:.4f}')

class ConvBNRelu(tf.keras.Model):
    def __init__(self, filters, kernel_size=3, strides=1, padding='SAME', weight_decay=0.0005, rate=0.4, drop=True):
        super(ConvBNRelu, self).__init__()
        self.drop = drop
        self.conv = keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=strides,
                                        padding=padding, kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.dropOut = keras.layers.Dropout(rate=rate)

    def call(self, inputs, training=False):
        layer = self.conv(inputs)
        layer = tf.nn.relu(layer)
        layer = self.batchnorm(layer)
        if self.drop:
            layer = self.dropOut(layer)

        return layer

class VGG16Model(tf.keras.Model):
    def __init__(self):
        super(VGG16Model, self).__init__()
        self.conv0 = ConvBNRelu(filters=64, kernel_size=[3, 3], rate=0.3)
        self.conv1 = ConvBNRelu(filters=64, kernel_size=[3, 3])
        self.conv2 = ConvBNRelu(filters=64, kernel_size=[3, 3], drop=False)
        self.maxPooling1 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv3 = ConvBNRelu(filters=128, kernel_size=[3, 3])
        self.conv4 = ConvBNRelu(filters=128, kernel_size=[3, 3], drop=False)
        self.maxPooling2 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv5 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv6 = ConvBNRelu(filters=256, kernel_size=[3, 3])
        self.conv7 = ConvBNRelu(filters=256, kernel_size=[3, 3], drop=False)
        self.maxPooling3 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv11 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv12 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv13 = ConvBNRelu(filters=512, kernel_size=[3, 3], drop=False)
        self.maxPooling5 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.conv14 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv15 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv16 = ConvBNRelu(filters=512, kernel_size=[3, 3])
        self.conv17 = ConvBNRelu(filters=512, kernel_size=[3, 3],drop=False)
        self.maxPooling6 = keras.layers.MaxPooling2D(pool_size=(2, 2))
        self.flat = keras.layers.Flatten()
        self.dropOut = keras.layers.Dropout(rate=0.5)
        self.dense1 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.dense11 = keras.layers.Dense(units=512,
                                         activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0005))
        self.dense2 = keras.layers.Dense(units=10)
        self.softmax = keras.layers.Activation('softmax')

    def call(self, inputs, training=False):
        net = self.conv0(inputs, training=training)
        net = self.conv1(net, training=training)
        net = self.conv2(net, training=training)
        net = self.maxPooling1(net)
        net = self.conv3(net, training=training)
        net = self.conv4(net, training=training)
        net = self.maxPooling2(net)
        net = self.conv5(net, training=training)
        net = self.conv6(net, training=training)
        net = self.conv7(net, training=training)
        net = self.maxPooling3(net)
        net = self.conv11(net, training=training)
        net = self.conv12(net, training=training)
        net = self.conv13(net, training=training)
        net = self.maxPooling5(net)
        net = self.conv14(net, training=training)
        net = self.conv15(net, training=training)
        net = self.conv16(net, training=training)
        net = self.conv17(net, training=training)
        net = self.maxPooling6(net)
        net = self.dropOut(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.dense11(net)
        net = self.dense2(net)
        net = self.softmax(net)
        return net


if __name__ == '__main__':
    print(tf.__version__)
    print(keras.__version__)

    training_epochs = 500
    batch_size = 128
    learning_rate = 0.1
    momentum = 0.9
    lr_decay = 1e-6
    lr_drop = 20

    tf.random.set_seed(777)

    def lr_scheduler(epoch):
        return learning_rate * (0.5 ** (epoch // lr_drop))

    reduce_lr = keras.callbacks.LearningRateScheduler(lr_scheduler)

    train_images, train_labels, val_images, val_labels, test_images, test_labels = load_images()


    # data augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(train_images)

    model = VGG16Model()

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,
                                        decay=1e-6, momentum=momentum, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    model.fit(datagen.flow(train_images, train_labels,
                        batch_size=batch_size), 
          epochs=training_epochs, 
          verbose=0,  # 출력 숨기기
          callbacks=[reduce_lr, EpochEndCallback((train_images, train_labels), (val_images, val_labels), (test_images, test_labels))],
          steps_per_epoch=train_images.shape[0] // batch_size,
          validation_data=(val_images, val_labels))

Physical devices cannot be modified after being initialized
2.10.0
2.10.0
Epoch 1/500 - Train Accuracy: 0.1664 - Val Accuracy: 0.1650 - Test Accuracy: 0.1615
Epoch 2/500 - Train Accuracy: 0.1834 - Val Accuracy: 0.1849 - Test Accuracy: 0.1831
Epoch 3/500 - Train Accuracy: 0.1884 - Val Accuracy: 0.1898 - Test Accuracy: 0.1888
Epoch 4/500 - Train Accuracy: 0.3582 - Val Accuracy: 0.3570 - Test Accuracy: 0.3488
Epoch 5/500 - Train Accuracy: 0.3687 - Val Accuracy: 0.3660 - Test Accuracy: 0.3714
Epoch 6/500 - Train Accuracy: 0.4013 - Val Accuracy: 0.4061 - Test Accuracy: 0.3997
Epoch 7/500 - Train Accuracy: 0.4692 - Val Accuracy: 0.4672 - Test Accuracy: 0.4602
Epoch 8/500 - Train Accuracy: 0.5501 - Val Accuracy: 0.5514 - Test Accuracy: 0.5443
Epoch 9/500 - Train Accuracy: 0.5276 - Val Accuracy: 0.5241 - Test Accuracy: 0.5156
Epoch 10/500 - Train Accuracy: 0.6087 - Val Accuracy: 0.6153 - Test Accuracy: 0.6017
Epoch 11/500 - Train Accuracy: 0.6676 - Val Accuracy: 0.6673 - Test Accuracy: 0.6611
